# Data Acquisition Example: Testing WLAN Client Receiver System Noise
_By Dan Kuester_

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [4]:
# This should be run in python 3.6.x from the computer connected to the AP side

%pylab inline
import time
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 20 # seconds
acquisition_time = 5 #  seconds

iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(list(range(20,61,40)),[110]+list(range(0,41,40)))
#np.random.shuffle(c_e0_points) # randomize?

# db_path = '{}'.format(time.strftime("%Y-%m-%d-%HH-%MM"))

lb.show_messages('debug')

## This `with' block ensures sockets and iperf subprocesses are
## closed correctly when the script ends, even if exceptions are raised
c = ssm.instruments.MiniCircuitsRCDAT('11604210008')
e0 = ssm.instruments.MiniCircuitsRCDAT('11604210014')
iperf_client = ssm.software.IPerf('10.0.0.3', bind='10.0.0.2', interval=0.1)
iperf_server = ssm.software.IPerf(bind='10.0.0.3')
wlan = ssm.software.WLANStatus('Wi-Fi', ssid='Engenius1')
db = lb.StatesToSQLite(time.strftime("%Y-%m-%d-%H%M%S"))

import time
t0 = time.time()
with lb.concurrently(c, e0, iperf_client, iperf_server, wlan, db):
    # Log all state changes in lte_laa and each client in iperf_clients
    db.observe_states([c,e0,iperf_server,iperf_client,wlan])

    # Only need to do this if there is a variable attenuator in the test setup
    # lte_enb_atten.state.attenuation = 20. # dB

    print('Pausing for {}s setup time'.format(setup_time))
    c.state.attenuation = 30
    e0.state.attenuation = 110
    iperf_server.start()
    iperf_client.start()
    time.sleep(setup_time)
    df=iperf_client.fetch()
    df_server=iperf_server.fetch()
    
#     row = {}
#     i = 0
#     for c.state.attenuation, e0.state.attenuation in sweep_points:
#         i += 1
#         print('{}/{}: atten C {} dB, atten E0 {} dB'\
#               .format(i, len(sweep_points), c.state.attenuation, e0.state.attenuation))
            
#         time.sleep(acquisition_time)
        
#         for sample,row in iperf_client.fetch().iterrows():
#             db.append(**row.to_dict())
#         db.write()

# # # Load the sqlite database and save a copy into a csv
# df = lb.read_sqlite(db_path)
# df.to_csv(db_path+'.csv')

2018-10-16 11:48:00.483 - DEBUG - WLANStatus('Wi-Fi') - starting WLAN reconnect watchdog


Populating the interactive namespace from numpy and matplotlib


2018-10-16 11:48:00.797 - DEBUG - MiniCircuitsRCDAT('11604210014') - Connected to MiniCircuits RCDAT-6000-110 attenuator, SN#11604210014
2018-10-16 11:48:00.809 - DEBUG - MiniCircuitsRCDAT('11604210008') - Connected to MiniCircuits RCDAT-6000-110 attenuator, SN#11604210008
2018-10-16 11:48:01.000 - INFO - Connected in 0.52s
2018-10-16 11:48:01.055 - DEBUG - MiniCircuitsRCDAT('11604210008') - set attenuation 30.0 dB
2018-10-16 11:48:01.067 - DEBUG - MiniCircuitsRCDAT('11604210014') - set attenuation 110.0 dB
2018-10-16 11:48:01.070 - DEBUG - IPerf('') - background execute: '..\\..\\..\\..\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\ssmdevices\\lib\\iperf.exe -n -1 -y C -s -B 10.0.0.3 -i 0.25 -p 5001 -t 10'
2018-10-16 11:48:01.124 - DEBUG - IPerf('10.0.0.3') - background execute: '..\\..\\..\\..\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\ssmdevices\\lib\\iperf.exe -n -1 -y C -c 10.0.0.3 -B 10.0.0.2 -i 0.1 -p 5001 -t 10'


Pausing for 20s setup time


2018-10-16 11:48:11.537 - DEBUG - IPerf('10.0.0.3') - stdout closed; execution done
2018-10-16 11:48:21.213 - DEBUG - Host('') - Host('') disconnected
2018-10-16 11:48:21.214 - DEBUG - MiniCircuitsRCDAT('11604210008') - MiniCircuitsRCDAT('11604210008') disconnected
2018-10-16 11:48:21.216 - DEBUG - MiniCircuitsRCDAT('11604210014') - MiniCircuitsRCDAT('11604210014') disconnected
2018-10-16 11:48:21.217 - WARNING - IPerf('10.0.0.3') - tried kill(), but no process is running
2018-10-16 11:48:21.218 - DEBUG - IPerf('10.0.0.3') - IPerf('10.0.0.3') disconnected
2018-10-16 11:48:21.229 - DEBUG - IPerf('') - stdout closed; execution done
2018-10-16 11:48:21.234 - DEBUG - IPerf('') - IPerf('') disconnected
2018-10-16 11:48:21.236 - WARNING - Netsh('') - tried kill(), but no process is running
2018-10-16 11:48:21.238 - DEBUG - Netsh('') - Netsh('') disconnected
2018-10-16 11:48:21.239 - DEBUG - WLANStatus('Wi-Fi') - WLANStatus('Wi-Fi') disconnected
2018-10-16 11:48:21.240 - INFO - Disconnected a

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\python code\\junk.csv'

In [8]:
iperf_client.settings.arguments

['-n', '-1', '-y', 'C']

In [3]:
import traitlets as tl
tr = lb.List(default_value=['hi'])
print('tr default is ', tr.default_value)

tr default is  ['hi']


In [4]:
tr = tl.List(trait=tl.Unicode(), default_value=['7'])
tr._valid_defaults

(list, tuple, set, frozenset)

In [5]:
tl.__version__

'4.3.2'